In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from lxml import etree, html as lhtml
from html.parser import HTMLParser
import requests
import re

In [2]:
author_to_id = {
    "Достоевский Ф. М.": 9150,
    "Роллинс Дж.": 59396,
    "Фицджеральд Ф. С.": 28727,
    "Глуховский Д. А.": 53427,
    "Стругацкий А. Н.": 26268,
    "Лукьяненко С. В.": 16626,
    "Фрай М.": 28927,
    "Хантер Э.": 37969,
    "Роулинг Дж. К.": 104832,
}

In [3]:
root = 'https://www.moscowbooks.ru/'
book_ids = []
for i in author_to_id:
    author_id = author_to_id[i]
    next_url = 'catalog/author/' + str(author_id)
    while next_url != '':
        curr_html = requests.get(root+next_url).text
        next_url = '';

        soup = BeautifulSoup(curr_html, 'lxml')

        prices = soup.find_all(class_='book-preview__buy-button button button_primary tocart_btn')
        for price in prices:
            book_ids.append(int(price['data-productid']))
    
        next_pages = soup.find_all(class_='pager__text', title=True)
        for next_page in next_pages:
            if next_page['title']=='следующая страница':
                next_url = next_page['href']
                break

In [4]:
RE_PRICE_REMOVE = re.compile(r'[^0-9]')
root = 'https://www.moscowbooks.ru/'

def extract_table(header):
    left = header.find(class_='book__details-left')
    right = header.find(class_='book__details-right')
    
    items_left = left.find_all('dl')
    items_right = right.find_all('dl')
    items = items_left + items_right
    d = {}
    for item in items:
        item_name = item.find(class_='book__details-name').text.strip()[:-1]
        item_value = item.find(class_='book__details-value').text.strip()
        d[item_name] = item_value
    return d

def get_description(description):
    description.find('b').replace_with('')
    description.find('a').replace_with('')
    for br in description.find_all("br"):
        br.replace_with("\n")
    return description.text.strip()
    
def extract_book_info(book_id, n_attempts=5):
    book_url = 'https://www.moscowbooks.ru/book/' + str(book_id)
    
    r = requests.get(book_url)
    count = 0
    while r.status_code != 200 and count < n_attempts:
        r = requests.get(book_url)
        count += 1
        
    book_html = r.text
    soup = BeautifulSoup(book_html, 'lxml')
    
    d = {}
    d['Код товара'] = book_id
    
    #author's name
    author_header = soup.find(class_='page-header__author')
    authors = ', '.join([i.text for i in author_header.find_all(class_='author-name')])
    d['Автор'] = authors
    
    #book's name
    name_header = soup.find(class_='page-header__title')
    name = re.sub(authors, '', name_header.text.strip())
    d['Название'] = name
    
    #img
    img_header = soup.find(class_='book__cover').find(class_='link_gallery')
    img_url = root[:-1] + img_header['href']
    d['Обложка'] = img_url
    
    #rating
    star_header = soup.find(class_='book___rating-stars rating-stars rating-stars_lg')
    stars = star_header['data-rate']
    d['Рейтинг'] = stars
    
    #stickers
    sticker_header = soup.find(class_='book__stickers stickers stickers_lg')
    stickers = []
    if sticker_header:
        for sticker_item in sticker_header.find_all(class_='stickers__item'):
            sticker_icon = sticker_item.find(class_='stickers__icon')
            sticker_label = sticker_item.find(class_='label')
            if sticker_icon:
                stickers.append(sticker_icon['title'])
            elif sticker_label:
                stickers.append(sticker_label.text)
        stickers = '; '.join((i for i in stickers))
        d['Стикеры'] = stickers
    
    #in_stock
    stock_header = soup.find(class_='instock1')
    in_stock = True if stock_header else False
    d['Наличие'] = in_stock
    
    #price
    price_header = soup.find(class_='book__price')
    price = RE_PRICE_REMOVE.sub('', price_header.text)
    d['Цена'] = price + ' р.'
    
    #table
    table_header = soup.find(class_='book__details-list js-book-details-list')
    table = extract_table(table_header)
    d.update(table)
    
    #description
    description_header = soup.select('div.book__description.js-book-description')
    description = get_description(description_header[0])
    d['Описание'] = description
    
    return d

У процессов отдельное адресное пространство, поэтому используем специальный объект Value, чтобы была возможность делить эту переменную со всеми процессами

In [5]:
from multiprocessing import Pool, Lock, Value
import time

mutex = Lock()
n_processed = Value('i', 0)

def func_wrapper(book_id):
    res = extract_book_info(book_id) 
    with mutex:
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return res

with Pool(processes=10) as pool:
    starttime = time.time()
    result = pool.map(func_wrapper, book_ids)
    print('\nThat took {} seconds'.format(time.time() - starttime))

240 objects are processed...
That took 346.9374928474426 seconds


In [6]:
assert(len(book_ids)==len(result))

In [7]:
df = pd.DataFrame(result)
df = df.replace(np.nan, '', regex=True)

df.sort_values(by=['Код товара'], inplace=True)
with open('hw_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)

In [8]:
pd.reset_option("^display")
pd.set_option('display.max_columns', 5000)

In [9]:
df.head()

,Код товара,Автор,Название,Обложка,Рейтинг,Стикеры,Наличие,Цена,Издательство,Год издания,Место издания,Язык текста,Тип обложки,Формат,Размеры в мм (ДхШхВ),Вес,Страниц,Тираж,Артикул,ISBN,В продаже с,Описание,Возраст,Тиснение,Бумага,Обрез,Иллюстрации,Язык оригинала,Перевод,Футляр,Производитель,Год производства,Место производства,Иллюстраторы
84,1000044,Фицджеральд Ф. С.,Великий Гэтсби = The Great Gatsby. Уникальная ...,https://www.moscowbooks.ru/image/book/664/w600...,0,,True,224 р.,АСТ,2019,Москва,английский,Мягкая обложка,60х84 1/16,200x137x24,260 гр.,320,2000 экз.,ASE000000000842319,978-5-17-114038-0,27.07.2019,Лучший способ учить иностранный язык — это чит...,,,,,,,,,,,,
2,1000435,Достоевский Ф. М.,Чужая жена и муж под кроватью.,https://www.moscowbooks.ru/image/book/664/w600...,0,,True,144 р.,АСТ,2019,Москва,русский,Мягкая обложка,76х100 1/32,180x115x27,240 гр.,384,5000 экз.,ASE000000000843846,978-5-17-115705-0,30.07.2019,Очень необычный сборник. Очень непривычный Дос...,,,,,,,,,,,,
119,1000624,"Стругацкий А. Н., Стругацкий Б. Н.",Полное собрание сочинений. В 33 томах. Том 16....,https://www.moscowbooks.ru/image/book/664/w600...,0,,True,3000 р.,Издательство Сидорович,2019,СПб,русский,Твердый переплет (ткань),84х108 1/32,208x132x24,490 гр.,384,350 экз.,978-5-905909-26-9,978-5-905909-26-9,31.07.2019,Полное собрание сочинений Аркадия и Бориса Стр...,,,,,,,,,,,,
120,1000625,"Стругацкий А. Н., Стругацкий Б. Н.",Полное собрание сочинений. В 33 томах. Том 17....,https://www.moscowbooks.ru/image/book/664/w600...,0,,True,3000 р.,Издательство Сидорович,2019,СПб,русский,Твердый переплет (ткань),84х108 1/32,208x132x38,760 гр.,624,350 экз.,978-5-905909-27-6,978-5-905909-27-6,31.07.2019,Полное собрание сочинений Аркадия и Бориса Стр...,,,,,,,,,,,,
156,1002585,Лукьяненко С. В.,Звезды — холодные игрушки. Звездная тень.,https://www.moscowbooks.ru/image/book/666/w600...,0,,True,566 р.,АСТ,2019,Москва,русский,Твердый переплет,60х90 1/16,218x143x37,655 гр.,672,2000 экз.,ASE000000000845549,978-5-17-117177-3,15.08.2019,Встреча с иными цивилизациями оказалась обеску...,,,,,,,,,,,,
